# TP contrôle: Développement à l'aide de l'objet d'un "scanneur" de port 

## Conditions et objets du TP

Ce TP est à faire individuellement, aucune forme d'échange avec un autre étudiant(e) ne sera tolérée.(Les smartphones et les montres communicantes seront rangés).
L'objet de ce TP est de développer sous la forme d'un ensemble d'objets un scanneur de ports.
Les développements n'utilisant pas des classes objets ne seront pas pris pris en compte pour la notation.
La qualité du code et l'environnement (typing, commentaires, gestion des exceptions, utilisation de vscode et de vim, qualité du code) seront pris en compte sur 5 points.

Le code est à rendre sur GitHub.

Le package python-nmap de Python est un wrapper Python des commandes nmap.

Installez les packages dnspython python-nmap via pip.
```bash
pip install --user dnspython python-nmap
```
Des exemples de résolution dns avec Python sont visibles [ici](https://www.dnspython.org/examples/)

## 1. Créer une classe Resolveur dont la signature est la suivante

```python

class Resolveur:
    """Class custom resolver
       ns: IP du dns qui fera la révolution """
    def __init__(self, ns: str = '1.1.1.1') -> str:
        ...
        
    def resoudInverse(self, mon_ipaddr: IPAddress) -> str:
        """prend une instance de IPAddress(NetAddr) et retourne un F.Q.D.N."""
        ...

# appel
mon_resolveur = Resolveur(ns='1.1.1.1') 
```
faites valider par l'enseignant son bon fonctionnement (check1)

In [15]:
import dns.resolver
import netaddr


class Resolveur:
    """Class custom resolver
       ns: IP du dns qui fera la révolution """
    def __init__(self, ns: str = '194.199.227.80') -> str:
        IPAddress= dns.resolver.resolve(ns,'A')
        for data in IPAddress:
            print ("L'adresse IP du réseau :", data)
    
    def resoudInverse(self, mon_ipaddr: IPAddress) -> str:
        """prend une instance de IPAddress(NetAddr) et retourne un F.Q.D.N."""
        fqdn=dns.resolver.resolve(mon_ipaddr,'ptr')
        for data in fqdn:
            print('Le nom du serveur est', data)
        
    

# appel
mon_resolveur = Resolveur(ns='www.iutbeziers.fr')
mon_resolveur = Resolveur(mon_ipaddr='146.59.209.152')



NXDOMAIN: The DNS query name does not exist: 194.199.227.80.

## 2. Implémentation objet de la classe scanner
### Package nmap
Le code suivant permet de faire un scan nmap en mode procédural. Le But est de produire un code objet équivalent.

In [5]:
import nmap
import netaddr
import logging


targets:tuple[str,...] = ('10.255.255.135','10.255.255.200')
dports:tuple[int,...] = (21,80,443,161,3306,4443,8080)
logging.getLogger("nmap").setLevel(logging.ERROR)

# create a port scanner object
scanner = nmap.PortScanner()
  
# scan the target ip and ports
for ip in targets:
    print("scan de l'IP: {}\n".format(ip))
    for port in dports:
        print(f"scan de du port: {port}\n")
        try:
            scanner.scan(ip, str(port))
            state=scanner[ip]['tcp'][port]['state']
            print("Port {}: {}".format(port, state))
            print(50*"=")
        except KeyError as e:            
            print("machine inaccessible")
            break
        


PortScannerError: 'nmap program was not found in path. PATH is : /bin:/home/lucky/.local/bin:/home/lucky/.pyenv/plugins/pyenv-virtualenv/shims:/home/lucky/.pyenv/shims:/home/lucky/.pyenv/bin:/home/lucky/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin'

### Ajoutez à ce code Objet une Dataclass ScanMachine chargée du scan d'une instance de la Classe Cible

```python
from dataclasses import dataclass
from typing import List, Tuple, Type 
import logging
import nmap
import dns.resolver
import socket
from netaddr import IPAddress

@dataclass
class Cible:
    """Cible à scanner (IPAddress fqdn port)"""
    ip_cible: IPAddress
    fqdn_cible: str
    ports_cible: Tuple 

@dataclass
class ScanResult:
    """ Résultat d'un scan pour un port et une machine donnée  """
    ip_cible: IPAddress
    fqdn_cible: str
    port_cible: Tuple 
    state_cible: str
    
    
    def __str__(self) -> str:
        return f"ScanResult(ip={self.ip_cible}, fqdn={self.fqdn_cible}, port={self.port_cible}, state={self.state_cible})\n_________________________________________________"

    def __repr__(self) -> str:
        return f"ScanResult(ip={self.ip_cible}, fqdn={self.fqdn_cible}, port={self.port_cible}, state={self.state_cible})\n_________________________________________________"
    


class Resolveur:
    """Class custom resolver"""

class ScanMachine:
    """Classe de scan"""

    def __init__(self, cible:type[Cible]):    
 
        

    def scan(self):
        """"scanne la cible"""
        # votre code ici

        ### Affichage du résultat avec la classe ScanResult
        print(ScanResult(self.ip_cible, self.fqdn_cible, port, self.state))
            
   

```
Faites vérifiez le bon fonctionnement de votre classe (Check2)

```Python
cibles: Tuple[str,...] = ('10.255.255.200','146.59.209.152', '194.199.227.136')
ports: Tuple[int,...] = (21, 22, 25, 53, 80, 161, 389, 443, 636, 3306, 6443, 8080)
ipAdresse: str

mon_resolveur = Resolveur(ns='1.1.1.1') 
for indice,ipAdresse in enumerate(cibles):
    fqdn = mon_resolveur.resoudInverse(IPAddress(ipAdresse))  
    t = Cible(ipAdresse, fqdn, ports)
    print(f"Cible {indice} : {ipAdresse} {fqdn} {ports}")
    s = ScanMachine(t)
```

3. Ajoutez une fonctionnalité de récupération des bannières des services scannés pour les ports 21,22,25,80 à l'aide du module Socket (check3)      
   
Résultats Attendus:

```bash
❯ python -u "/home/pouchou/ownCloud/formations/DEV/R308/R308-NOTEBOOKS/CM-TD-TP/TP/TP4-controle/scan-objet-banniere2.py"
Cible 0 : 10.255.255.200 fqdn inconnu (21, 22, 25, 53, 80, 161, 389, 443, 636, 3306, 6443, 8080)
scan du port: 21

port 21 is open with banner 220 ProFTPD Server (IUT BEZIERS) [10.255.255.200]

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=21, state=open, bannière=220 ProFTPD Server (IUT BEZIERS) [10.255.255.200]
)
_________________________________________________
scan du port: 22

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=22, state=filtered, bannière=220 ProFTPD Server (IUT BEZIERS) [10.255.255.200]
)
_________________________________________________
scan du port: 25

port 25 is open with banner 220 mail.iutbeziers.fr ESMTP Exim 4.82 Ubuntu Sat, 15 Oct 2022 22:08:55 +0200

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=25, state=open, bannière=220 mail.iutbeziers.fr ESMTP Exim 4.82 Ubuntu Sat, 15 Oct 2022 22:08:55 +0200
)
_________________________________________________
scan du port: 53

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=53, state=open, bannière=)
_________________________________________________
scan du port: 80

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=80, state=open, bannière=inconnue)
_________________________________________________
scan du port: 161

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=161, state=filtered, bannière=inconnue)
_________________________________________________
scan du port: 389

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=389, state=open, bannière=)
_________________________________________________
scan du port: 443

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=443, state=open, bannière=)
_________________________________________________
scan du port: 636

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=636, state=open, bannière=)
_________________________________________________
scan du port: 3306

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=3306, state=filtered, bannière=)
_________________________________________________
scan du port: 6443

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=6443, state=filtered, bannière=)
_________________________________________________
scan du port: 8080

ScanResult(ip=10.255.255.200, fqdn=fqdn inconnu, port=8080, state=filtered, bannière=)
_________________________________________________
Cible 1 : 146.59.209.152 cluster031.hosting.ovh.net. (21, 22, 25, 53, 80, 161, 389, 443, 636, 3306, 6443, 8080)
scan du port: 21

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=21, state=filtered, bannière=)
_________________________________________________
scan du port: 22

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=22, state=filtered, bannière=)
_________________________________________________
scan du port: 25

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=25, state=filtered, bannière=)
_________________________________________________
scan du port: 53

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=53, state=filtered, bannière=)
_________________________________________________
scan du port: 80

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=80, state=open, bannière=inconnue)
_________________________________________________
scan du port: 161

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=161, state=filtered, bannière=inconnue)
_________________________________________________
scan du port: 389

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=389, state=filtered, bannière=inconnue)
_________________________________________________
scan du port: 443

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=443, state=open, bannière=)
_________________________________________________
scan du port: 636

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=636, state=filtered, bannière=)
_________________________________________________
scan du port: 3306

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=3306, state=filtered, bannière=)
_________________________________________________
scan du port: 6443

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=6443, state=filtered, bannière=)
_________________________________________________
scan du port: 8080

ScanResult(ip=146.59.209.152, fqdn=cluster031.hosting.ovh.net., port=8080, state=filtered, bannière=)
_________________________________________________
Cible 2 : 194.199.227.136 wscodoc.iutbeziers.fr. (21, 22, 25, 53, 80, 161, 389, 443, 636, 3306, 6443, 8080)
scan du port: 21

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=21, state=filtered, bannière=)
_________________________________________________
scan du port: 22

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=22, state=filtered, bannière=)
_________________________________________________
scan du port: 25

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=25, state=filtered, bannière=)
_________________________________________________
scan du port: 53

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=53, state=filtered, bannière=)
_________________________________________________
scan du port: 80

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=80, state=open, bannière=inconnue)
_________________________________________________
scan du port: 161

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=161, state=filtered, bannière=inconnue)
_________________________________________________
scan du port: 389

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=389, state=filtered, bannière=inconnue)
_________________________________________________
scan du port: 443

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=443, state=open, bannière=)
_________________________________________________
scan du port: 636

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=636, state=filtered, bannière=)
_________________________________________________
scan du port: 3306

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=3306, state=filtered, bannière=)
_________________________________________________
scan du port: 6443

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=6443, state=filtered, bannière=)
_________________________________________________
scan du port: 8080

ScanResult(ip=194.199.227.136, fqdn=wscodoc.iutbeziers.fr., port=8080, state=filtered, bannière=)
_________________________________________________
                                                                                        ─╯
```
